## 2 Assignment
## 다음과 같은 CNN 모델을 작성해보자

* Input
  * Input type: torch.Tensor  
  * Input shape: (?, 1, 28, 28)
    * 여러장의, 흑백, 28x28 size의 이미지라고 가정하자
* Layers  
  * Layer 1
    * Conv2d >> C: 32, Kernel size (필터 크기): 3, Stride: 1, Padding: 1  
    * ReLu  
    * MaxPool >> Kernel size: 2, Stride: 2
    * 입-출력 (?, 1, 28, 28) >> (?, 32, 14, 14)
  * Layer 2
    * Conv2d >> C: 64, Kernel size (필터 크기): 3, Stride: 1, Padding: 1
    * ReLU
    * MaxPool >> Kernel size: 2, Stride: 2
    * 입-출력 (?, 32, 14, 14) >> (?, 64, 7, 7)
  * Layer 3
    * Conv2d >> C: 128, Kernel size (필터 크기): 3, Stride: 1, Padding: 1
    * ReLU
    * MaxPool >> Kernel size: 2, Stride: 2, Padding: 1
    * 입-출력 (?, 64, 7, 7) >> (?, 128, 4, 4)
  * Layer 4
    * Linear >> input: 4x4x128 output: 625
    * ReLU
    * Dropout
    * 입-출력 (4x4x128) >> (625)
  * Layer 5
    * Linear >> input: 625 output: 10
    * Softmax (pytorch의 Cross Entropy Loss 함수를 사용하는 것을 감안한다)

In [ ]:
# Pytorch, Dataset을 관리하는 torchvsion.datasets, 데이터셋을 변형하는 torchvision.transforms, 뉴럴 네트워크를 위한 nn 이 임포트 되었다.


import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

import numpy as np
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as opt

In [ ]:
# Cuda라는 장치가 연산처리가 가능하면 Cuda를 사용한다. 이외에는 cpu로 연산을 처리한다.(Cuda=GPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility, 학습결과를 어느수준 일정하게 만들기 위하여 시드를 부여한다. Cuda를 사용하는 경우 Cuda에도 시드를 부여해줘야한다.
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
# CNN을 사용하기 위해 클래스를 정의한다.
# Sequential 함수를 통해 여러 연속적인 동작을 하나의 Layer로 묶을 수 있다.
# Conv2d는 2차원 모델을 convolution 하는 함수 (N은 배치사이즈,  커널 사이즈 = 필터 사이즈, stride, padding 지정가능)
# ReLu는 다루지 않는다.
# MaxPool2d는 2차원의 모델을 Max-Pooling하는 함수
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # L1 Input shape=(?, 1, 28, 28)
        #    Conv     -> (?, 32, 28, 28)
        #    Pool     -> (?, 32, 14, 14)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 Input shape=(?, 32, 14, 14)
        #    Conv      ->(?, 64, 14, 14)
        #    Pool      ->(?, 64, 7, 7)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L3 Input shape=(?, 64, 7, 7)
        #    Conv      ->(?, 128, 7, 7)
        #    Pool      ->(?, 128, 4, 4) (padding 1)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))
        # L4 Linear 128x4x4 inputs -> 625 outputs
        self.layer4 = torch.nn.Sequential(
            torch.nn.Linear(128*4*4, 625, bias=True),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.2)  #Dropout이란 뉴럴네트워크의 경로 몇개를 버리면서 학습하는 것으로, overfitting을 방지하는 효과를 가진다.
        )
        # L5 Linear 625 inputs -> 10 outputs
        self.layer5 = torch.nn.Linear(625, 10, bias=True)
        # 자비에르 유니폼: Linear의 초깃값을 0이아닌 자비에르방식으로 설정.
        torch.nn.init.xavier_uniform_(self.layer5.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC , Linear연산을 하기 전에 한줄 벡터로 변환하여 입력한다.
        out = self.layer5(out)
        return out